In [ ]:
import argparse
import networkx as nx
import numpy as np
import tensorflow as tf
import my
import objectives
import partition as pttn

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.enable_eager_execution(config)

In [ ]:
args = argparse.Namespace()
args.device = '/cpu:0'
args.graph = 'soc-Epinions1'
args.lambda_node = 1.0
args.lambda_edge = 1.0
args.lambda_entropy = 1.0
args.partitioner = 'greedy'
# args.partitioner = 'random'
args.n_machines = 10

In [ ]:
g = my.read_edgelist('data/' + args.graph, fill=True).to_directed()
adj = nx.adj_matrix(g)

In [ ]:
rng = np.random.RandomState(42)
partitioner = {
    'greedy' : pttn.GreedyVertexCut,
    'random' : pttn.RandomVertexCut,
}[args.partitioner](adj, args.n_machines, rng)

In [ ]:
partitioner.partition()
z = tf.one_hot(tf.constant(partitioner.eassign), args.n_machines)

In [ ]:
np.max(np.abs(partitioner.eassign[::2] - partitioner.eassign[1::2]))

In [ ]:
with tf.device(args.device):
    objective = objectives.Objective(adj, args.n_machines, args.lambda_node, args.lambda_edge, args.lambda_entropy)

In [ ]:
with tf.device(args.device):
    rslt, r, b_node, b_edge, entropy = objective(z, training=True)
    r, b_node, b_edge, entropy = objective(z)